In [1]:
# Import the data

import pandas as pd

df = pd.read_csv('../data/priest_popular_archetype_decks.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35867 entries, 0 to 35866
Columns: 1275 entries, deck_archetype to Murloc Holmes
dtypes: int64(1274), object(1)
memory usage: 348.9+ MB


In [2]:
df.head()

,deck_archetype,Circle of Healing,Flash Heal,Northshire Cleric,Power Word: Shield,Embrace the Shadow,Mind Blast,Shadow Word: Death,Shadow Word: Pain,Auchenai Soulpriest,...,Coilfang Constrictor,Snapdragon,Neptulon the Tidehunter,Ozumat,Prince Renathal,Ethereal Augmerchant,Replicat-o-tron,Cathedral of Atonement,Dispossessed Soul,Murloc Holmes
0,Control Priest,2,2,2,2,2,2,1,1,2,...,0,0,0,0,0,0,0,0,0,0
1,Dragon Priest,0,0,2,2,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Control Priest,2,0,2,2,0,0,2,0,2,...,0,0,0,0,0,0,0,0,0,0
3,Dragon Priest,0,0,2,2,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
4,C'Thun Priest,0,0,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Prepare Input (X) and Output (y)
X = df.drop(columns=['deck_archetype'])
y = df['deck_archetype']

In [4]:
# Encode Labels (deck_archetype)

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [5]:
# Split the data for training

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.25, random_state=42)

In [11]:
# Build Neural Network Model
# Note: have to use python 3.12.8 because tensorflow does not yet handle versions above this one.

from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.3),  # Prevent overfitting
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(len(encoder.classes_), activation='softmax')  # Multi-class output
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
# Model training

model.fit(X_train, y_train, epochs=200, validation_split=0.25, batch_size=8)

Epoch 1/200
2522/2522 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.5794 - loss: 1.2792 - val_accuracy: 0.7222 - val_loss: 0.7811
Epoch 2/200
2522/2522 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7188 - loss: 0.8230 - val_accuracy: 0.7359 - val_loss: 0.7443
Epoch 3/200
2522/2522 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7427 - loss: 0.7452 - val_accuracy: 0.7364 - val_loss: 0.7495
Epoch 4/200
2522/2522 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7565 - loss: 0.6820 - val_accuracy: 0.7359 - val_loss: 0.7558
Epoch 5/200
2522/2522 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7645 - loss: 0.6579 - val_accuracy: 0.7367 - val_loss: 0.7725
Epoch 6/200
2522/2522 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7780 - loss: 0.6147 - val_accuracy: 0.7361 - val_loss: 0.7833
Epoch 7/200
2522/2522 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7834 - loss: 0.5931 - val_accuracy: 0.7322 - val_loss: 0.8143
Epoch 8/200
2522/2522 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7884 - loss: 0

In [13]:
# Model evaluation

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

281/281 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6880 - loss: 3.6806
Test Accuracy: 0.70


In [18]:
# Export the model

model.save('../models/fnn_dense_layers_model.keras')